#### Splitting dataset

In [2]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd
from rdkit.Chem import rdMolDescriptors as rdescriptors
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys    
import pubchempy as pcp
import warnings
warnings.filterwarnings('ignore')
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')
from sklearn.utils import shuffle
import sys
sys.path.insert(0,"./../fip3")
from fip.profiles import CooccurrenceProfile
from fip import profiles
import re
from itertools import chain
from ast import literal_eval

In [3]:
#Loading database from csv 
suppl_coco = pd.read_csv('../data/CPF/COCONUT/COCOrandomF.csv', delimiter = ' ')

In [4]:
suppl_coco

,Coconut_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset,Fragments
0,CNP0239046,CCCCCC1C(=O)CCC1CC(=O)OC,226.156895,2.72510,KVWWIYGFBYDJQC-UHFFFAOYSA-N,KVWWIYGFBYDJQC,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 142, 195, 226, 294, 314, 334, 339, 370, 5...","'[CH][CH2][CH2][C]([CH])=[O]', '[CH][CH2][C](=..."
1,CNP0205086,Cc1c2c(cc3c1oc(=O)c1ccccc13)CN(c1cccc(Cl)c1)CO2,377.081871,5.26452,DWNKYEMDZRSBHK-UHFFFAOYSA-N,DWNKYEMDZRSBHK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[133, 137, 165, 216, 252, 314, 350, 382, 398, ...","'[c][c]([CH2])[cH][c]([c])[c]', '[cH][cH][cH][..."
2,CNP0361840,COc1cc(C2CC(=O)Oc3ccc4c(=O)c(-c5ccccc5)coc4c32...,458.136553,4.92680,ZRAWZNDFONIPGO-UHFFFAOYSA-N,ZRAWZNDFONIPGO,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[45, 60, 133, 145, 222, 227, 231, 249, 282, 28...","'[c][c]([c])[O][CH3]', '[c][cH][cH][c]([c])[c]..."
3,CNP0264800,C=C1C(=O)OC2C=C(CO)CCC(OCC)C3=CC(OC3=O)C12,320.125988,1.05360,PTJPAEBQCZMNAV-UHFFFAOYSA-N,PTJPAEBQCZMNAV,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[35, 80, 176, 222, 294, 314, 408, 514, 650, 65...","'[CH2][C]([CH2])=[CH][CH]([CH])[O]', '[C][C](=..."
4,CNP0396333,CC(=O)C(CC=C(C)C)C(C)=CCOc1ccc(CCNC(=O)c2ccccc...,419.246044,5.54570,FGXBFPFGWRVMGG-UHFFFAOYSA-N,FGXBFPFGWRVMGG,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 8, 80, 102, 197, 265, 310, 322, 367, 389, ...","'[cH][cH][cH][cH][cH]', '[c][CH2][CH2][NH][C]'..."
...,...,...,...,...,...,...,...,...,...,...
386292,CNP0315857,COC(=O)C1=COC(OC2OC(COC(=O)c3ccc(O)c(OC)c3)C(O...,554.163556,-0.63720,LIVSNGMPJUCRLO-UHFFFAOYSA-N,LIVSNGMPJUCRLO,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[58, 80, 145, 147, 167, 231, 233, 249, 314, 42...","'[CH]=[C]([CH])[C](=[O])[O][CH3]', '[CH2][CH](..."
386293,CNP0032553,Cc1c(CCC(=O)NCC(=O)NCC(=O)[O-])c(=O)oc2cc(O)c(...,395.065152,-0.62468,SSZVGMXOKPZNCV-UHFFFAOYSA-M,SSZVGMXOKPZNCV,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 80, 197, 252, 254, 305, 314, 366, 406, 42...","'[NH][CH2][C](=[O])[O-]', '[c][cH][c]([Cl])[c]..."
386294,CNP0223192,CC(=O)Oc1c(C)cc2c(c1O)C(=O)C(C1=CC(=O)c3cc(C)c...,490.089997,2.87624,JODPOMYHTFZNTK-UHFFFAOYSA-N,JODPOMYHTFZNTK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[16, 39, 84, 202, 224, 314, 333, 352, 516, 641...","'[c][cH][c]([CH3])[c]([c])[O]', '[C][C](=[CH])..."
386295,CNP0356727,C[N+]([O-])=NCOC1OC(CO)C(O)C(OC2OC(COC3OC(CO)C...,576.201398,-6.99890,STHHNNUYQQUJGY-UHFFFAOYSA-N,STHHNNUYQQUJGY,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[77, 80, 170, 222, 349, 443, 489, 588, 610, 65...","'[CH2][CH]([CH])[O][CH]([CH])[O]', '[CH][O][CH..."


In [5]:
suppl_coco = suppl_coco[['Fragments']]

In [6]:
suppl_coco

,Fragments
0,"'[CH][CH2][CH2][C]([CH])=[O]', '[CH][CH2][C](=..."
1,"'[c][c]([CH2])[cH][c]([c])[c]', '[cH][cH][cH][..."
2,"'[c][c]([c])[O][CH3]', '[c][cH][cH][c]([c])[c]..."
3,"'[CH2][C]([CH2])=[CH][CH]([CH])[O]', '[C][C](=..."
4,"'[cH][cH][cH][cH][cH]', '[c][CH2][CH2][NH][C]'..."
...,...
386292,"'[CH]=[C]([CH])[C](=[O])[O][CH3]', '[CH2][CH](..."
386293,"'[NH][CH2][C](=[O])[O-]', '[c][cH][c]([Cl])[c]..."
386294,"'[c][cH][c]([CH3])[c]([c])[O]', '[C][C](=[CH])..."
386295,"'[CH2][CH]([CH])[O][CH]([CH])[O]', '[CH][O][CH..."


In [7]:
df_split = np.array_split(suppl_coco, 10)

### Coocurence profile #0

In [10]:
df_split[0] = df_split[0] [['Fragments']]

In [11]:
df_split[0]

,Fragments
0,"'[CH][CH2][CH2][C]([CH])=[O]', '[CH][CH2][C](=..."
1,"'[c][c]([CH2])[cH][c]([c])[c]', '[cH][cH][cH][..."
2,"'[c][c]([c])[O][CH3]', '[c][cH][cH][c]([c])[c]..."
3,"'[CH2][C]([CH2])=[CH][CH]([CH])[O]', '[C][C](=..."
4,"'[cH][cH][cH][cH][cH]', '[c][CH2][CH2][NH][C]'..."
...,...
38625,'[cH][c]([cH])[C]1([O][CH])[O][C]1([CH3])[CH]'...
38626,"'[C][c]([c])[cH][cH][c]', '[CH2][CH]([CH])[O][..."
38627,"'[cH][c](=[O])[o][c]([c])[c]', '[c][cH][c]([cH..."
38628,"'[c][C](=[O])[O][CH]([CH2])[C]', '[CH]=[C]([CH..."


In [14]:
feature_list0 = []
for x in df_split[0]['Fragments']:
    tup = literal_eval(x)
    feature_list0.append(tup)
feature_list0[-1]

('[c][c]([c])[O][CH3]',
 '[c][cH][c]([OH])[c]([cH])[O]',
 '[c][c]([O])[c]([OH])[c]([c])[c]',
 '[c][cH][cH][c]([c])[O]',
 '[c][c](=[O])[cH][c]([O])[o]',
 '[cH][c]([O])[o][c]([c])[cH]',
 '[cH][cH][c]([O][CH3])[c]([cH])[OH]',
 '[cH][c]([c])[O][CH3]',
 '[c][O][c]([cH][c])[o][c]',
 '[c][c]([OH])[cH][c]([c])[o]',
 '[c][c]([OH])[cH][c]([cH])[O]',
 '[c][cH][c]([OH])[c]([c])[O]',
 '[c][cH][c]([o][c])[c]([c])[c]',
 '[cH][c]([cH])[O][c]([cH])[o]',
 '[c][O][c]([cH][cH])[cH][c]',
 '[c][cH][c](=[O])[c]([c])[c]',
 '[c][c]([OH])[c]([O][CH3])[c]([cH])[OH]',
 '[c][c]([OH])[c]([c]([cH])=[O])[c]([cH])[o]',
 '[c][cH][cH][c]([cH])[O]')

In [17]:
#Subset #0
p0 = CooccurrenceProfile.from_feature_lists(feature_list0)

In [18]:
p0.df

value
feature1                     feature2                                             
[CH2][CH2][CH2][CH2][CH]     [CH2][CH2][CH2][CH2][CH]                         1581
                             [CH2][CH2][CH]([C]([CH2])=[O])[CH]([CH2])[CH2]      2
                             [CH2][CH2][C](=[O])[CH]([CH2])[CH]                 17
                             [CH3][CH2][CH2][CH2]                              746
                             [CH3][CH2][CH2][CH2][CH2]                         718
...                                                                            ...
[c][c](=[O])[cH][c]([O])[o]  [c][c]([OH])[c]([O][CH3])[c]([cH])[OH]              1
                             [c][c]([OH])[c]([c]([cH])=[O])[c]([cH])[o]          1
                             [c][c]([O])[c]([OH])[c]([c])[c]                     1
                             [c][c]([c])[O][CH3]                                 1
[c][c]([OH])[cH][c]([cH])[O] [c][c]([OH])[c]([O][CH3])[c]([cH])[OH]              1

[4821131 rows x 1 columns]

In [19]:
p0.df.to_csv('../data/CPF/COCONUT/COCOp0_38630.csv')

In [20]:
p0 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp0_38630.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38630)
p0.df

value
feature1                     feature2                                             
[CH2][CH2][CH2][CH2][CH]     [CH2][CH2][CH2][CH2][CH]                         1581
                             [CH2][CH2][CH]([C]([CH2])=[O])[CH]([CH2])[CH2]      2
                             [CH2][CH2][C](=[O])[CH]([CH2])[CH]                 17
                             [CH3][CH2][CH2][CH2]                              746
                             [CH3][CH2][CH2][CH2][CH2]                         718
...                                                                            ...
[c][c](=[O])[cH][c]([O])[o]  [c][c]([OH])[c]([O][CH3])[c]([cH])[OH]              1
                             [c][c]([OH])[c]([c]([cH])=[O])[c]([cH])[o]          1
                             [c][c]([O])[c]([OH])[c]([c])[c]                     1
                             [c][c]([c])[O][CH3]                                 1
[c][c]([OH])[cH][c]([cH])[O] [c][c]([OH])[c]([O][CH3])[c]([cH])[OH]              1

[4821131 rows x 1 columns]

In [ ]:
p.add_another_cooccurrence_profile(p)

In [21]:
p0.add_another_cooccurrence_profile(p0)

### Coocurence profile #1

In [32]:
df_split[1] = df_split[1] [['Fragments']]
df_split[1]

,Fragments
38630,'[C][CH]([CH2])[C]([CH3])([CH2][CH2])[CH]([CH2...
38631,"'[CH][c]([c])[c]([O][CH2])[c]([cH])[CH2]', '[C..."
38632,"'[cH][cH][cH][cH][cH]', '[C][CH2][c]([cH][cH])..."
38633,"'[cH][cH][cH][o][c]', '[c][cH][cH][cH][o]', '[..."
38634,"'[CH][CH]=[CH][C](=[CH])[CH3]', '[CH3][C](=[O]..."
...,...
77255,"'[cH][cH][cH][cH][cH]', '[c][CH2][NH][C]([CH2]..."
77256,"'[cH][cH][cH][cH][c]', '[C][C]([CH2])([CH])[CH..."
77257,'[CH2][CH]([C])[C]([CH3])([CH3])[CH]([CH])[O]'...
77258,"'[CH2][CH]([CH])[O][CH]([CH])[O]', '[CH][CH2][..."


In [33]:
feature_list1 = []
for x in df_split[1]['Fragments']:
    tup = literal_eval(x)
    feature_list1.append(tup)
feature_list1[-1]

('[CH2][CH2][CH2][CH]([CH2])[O]',
 '[CH2][CH2][CH2][CH2][CH]',
 '[CH3][C](=[O])[O][CH]([CH2])[CH2]',
 '[CH3][CH2][CH2][CH2]',
 '[C][CH2][CH]([CH2][CH2])[O][C]',
 '[CH2][CH2][CH2][CH2][CH2]',
 '[CH][O][C]([CH3])=[O]',
 '[CH3][CH2][CH2][CH2][CH2]',
 '[CH2][CH]([O])[CH2][C](=[O])[OH]',
 '[CH][CH2][C](=[O])[OH]')

In [34]:
p1 = CooccurrenceProfile.from_feature_lists(feature_list1)
p1.df

value
feature1                            feature2                                                
[CH2][CH2][CH2][C]([CH])([CH])[CH3] [CH2][CH2][CH2][C]([CH])([CH])[CH3]                   96
                                    [CH2][CH2][C]([CH3])([C](=[O])[OH])[CH]([CH2])[C]     14
                                    [CH2][CH2][O][CH]([CH])[O]                             1
                                    [CH2][CH]([CH])[O][CH]([CH])[O]                        4
                                    [CH2][CH]([O])[CH]([OH])[CH]([CH])[OH]                 3
...                                                                                      ...
[C][NH][CH]([CH2][O])[CH]([CH2])[O] [C][O][CH]([CH]([CH])[O])[CH]([O])[O]                  1
[C][O][CH]([CH2][CH2])[C]([NH])=[O] [C][O][CH]([CH]([CH])[O])[CH]([CH])[O]                 1
                                    [C][O][CH]([CH]([CH])[O])[CH]([O])[O]                  1
[CH2][CH]([O])[CH2][C](=[O])[OH]    [CH3][C](=[O])[O][CH]([CH2])[CH2]                      1
[CH3][C](=[O])[O][CH]([CH2])[CH2]   [CH][CH2][C](=[O])[OH]                                 1

[4808910 rows x 1 columns]

In [35]:
p1.df.to_csv('../data/CPF/COCONUT/COCOp1_38630.csv')

In [36]:
p1 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp1_38630.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38630)
p1.df

value
feature1                            feature2                                                
[CH2][CH2][CH2][C]([CH])([CH])[CH3] [CH2][CH2][CH2][C]([CH])([CH])[CH3]                   96
                                    [CH2][CH2][C]([CH3])([C](=[O])[OH])[CH]([CH2])[C]     14
                                    [CH2][CH2][O][CH]([CH])[O]                             1
                                    [CH2][CH]([CH])[O][CH]([CH])[O]                        4
                                    [CH2][CH]([O])[CH]([OH])[CH]([CH])[OH]                 3
...                                                                                      ...
[C][NH][CH]([CH2][O])[CH]([CH2])[O] [C][O][CH]([CH]([CH])[O])[CH]([O])[O]                  1
[C][O][CH]([CH2][CH2])[C]([NH])=[O] [C][O][CH]([CH]([CH])[O])[CH]([CH])[O]                 1
                                    [C][O][CH]([CH]([CH])[O])[CH]([O])[O]                  1
[CH2][CH]([O])[CH2][C](=[O])[OH]    [CH3][C](=[O])[O][CH]([CH2])[CH2]                      1
[CH3][C](=[O])[O][CH]([CH2])[CH2]   [CH][CH2][C](=[O])[OH]                                 1

[4808910 rows x 1 columns]

### Coocurence profile 2

In [8]:
df_split[2] = df_split[2] [['Fragments']]
df_split[2]

,Fragments
77260,"'[cH][cH][c]([OH])[cH][cH]', '[C][c]([c])[c]([..."
77261,'[c][C]([CH2])([CH3])[CH]([CH2][CH2])[C](=[C])...
77262,"'[cH][cH][cH][cH][cH]', '[c][cH][c]([O][CH3])[..."
77263,"'[c][CH2][N]([CH2][CH])[CH]([C])[CH2]', '[c][c..."
77264,"'[C][c]([c])[cH][c]([cH])[OH]', '[c][cH][cH][c..."
...,...
115885,"'[c][cH][cH][c]([c])[n]', '[cH][cH][c]([n][c])..."
115886,"'[C][CH2][CH2][CH2][CH]', '[c][c]([c])[O][CH3]..."
115887,"'[c][c]([c])[O][CH3]', '[c][c]([CH2])[cH][c]([..."
115888,"'[cH][c](=[O])[o][c]([cH])[c]', '[c][c]([c])[O..."


In [9]:
feature_list2 = []
for x in df_split[2]['Fragments']:
    tup = literal_eval(x)
    feature_list2.append(tup)
feature_list2[-1]

('[CH]=[C]([CH])[C]([CH3])([CH2][CH2])[C]([CH])([CH2])[CH3]',
 '[C]=[CH][CH2][CH]([C])[C]',
 '[C][CH2][CH2][C]([CH3])([CH3])[CH2]',
 '[CH][O][CH2][CH]([CH])[O]',
 '[CH][O][CH]([O][CH2])[CH]([CH])[OH]',
 '[C][O][CH]([O][CH])[CH]([CH])[OH]',
 '[CH][CH]([OH])[CH]([OH])[CH2][O]',
 '[CH2][O][CH]([O][CH])[CH]([CH])[OH]',
 '[CH][O][CH]([CH2][O])[CH]([CH])[OH]',
 '[C][CH]([CH2])[C]([CH2][CH2])([CH2][CH2])[C](=[O])[O]',
 '[CH][O][CH]([CH2][OH])[CH]([CH])[OH]',
 '[CH][CH2][CH2][C]([CH])([CH3])[CH]',
 '[C][CH2][CH2][CH]([C])[C]',
 '[CH2][CH]=[C]([CH]([C])[CH2])[C]([CH2])([CH3])[C]',
 '[CH][CH]([CH])[O][CH]([CH])[O]',
 '[C][C]([CH2])([CH3])[C]([CH3])([CH2][CH2])[CH]([CH2])[C]',
 '[CH][CH]([OH])[CH]([OH])[CH]([CH])[OH]',
 '[CH][O][CH]([CH]([CH])[O])[CH]([CH])[OH]',
 '[C][CH2][CH2][CH]([C])[O]',
 '[CH][O][C](=[O])[C]([CH2])([CH2])[CH]',
 '[CH2][CH]([CH])[O][CH]([CH])[O]',
 '[C][C]([CH2])([CH])[CH2][CH2][C]',
 '[C][CH]([CH2])[C]([CH3])([CH3])[CH]([CH2])[O]',
 '[C][C](=[O])[O][CH]([CH])[O]',
 '[CH][CH

In [10]:
p2 = CooccurrenceProfile.from_feature_lists(feature_list2)
p2.df

value
feature1                               feature2                                     
[C][c]([c])[c]([OH])[c]([c])[CH3]      [C][c]([c])[c]([OH])[c]([c])[CH3]         154
                                       [C][c]([c])[c]([cH][c])[O][CH]              4
                                       [cH][cH][c]([OH])[cH][cH]                   9
                                       [cH][cH][c]([cH][cH])[CH]([CH])[O]          2
                                       [cH][c]([O])[c]([CH3])[c]([c])[OH]         22
...                                                                              ...
[c][cH][c]([CH3])[c]([c])[cH]          [c][c]([c])[O][CH3]                         1
[c][cH][c]([O][CH3])[c]([cH])[CH2]     [c][cH][c]([O][CH3])[c]([c])[O]             1
                                       [c][c]([c])[O][CH3]                         1
[c][cH][c]([O][CH3])[c]([c])[O]        [c][cH][c]([c]([cH])[CH3])[c]([cH])[o]      1
[c][cH][c]([c]([cH])[CH3])[c]([cH])[o] [c][c]([c])[O][CH3]                         1

[4760261 rows x 1 columns]

In [11]:
p2.df.to_csv('../data/CPF/COCONUT/COCOp2_38630.csv')

In [12]:
p2 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp2_38630.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38630)
p2.df

value
feature1                               feature2                                     
[C][c]([c])[c]([OH])[c]([c])[CH3]      [C][c]([c])[c]([OH])[c]([c])[CH3]         154
                                       [C][c]([c])[c]([cH][c])[O][CH]              4
                                       [cH][cH][c]([OH])[cH][cH]                   9
                                       [cH][cH][c]([cH][cH])[CH]([CH])[O]          2
                                       [cH][c]([O])[c]([CH3])[c]([c])[OH]         22
...                                                                              ...
[c][cH][c]([CH3])[c]([c])[cH]          [c][c]([c])[O][CH3]                         1
[c][cH][c]([O][CH3])[c]([cH])[CH2]     [c][cH][c]([O][CH3])[c]([c])[O]             1
                                       [c][c]([c])[O][CH3]                         1
[c][cH][c]([O][CH3])[c]([c])[O]        [c][cH][c]([c]([cH])[CH3])[c]([cH])[o]      1
[c][cH][c]([c]([cH])[CH3])[c]([cH])[o] [c][c]([c])[O][CH3]                         1

[4760261 rows x 1 columns]

### Coocurence profile 3

In [13]:
df_split[3] = df_split[3] [['Fragments']]
df_split[3]

,Fragments
115890,"'[c][c]([c])[O][CH3]', '[cH][c]([O])[c]([OH])[..."
115891,"'[cH][cH][c]([OH])[cH][cH]', '[c][O][CH]([c]([..."
115892,"'[N][O][CH2][C]([NH])=[O]', '[CH2][CH2][NH][C]..."
115893,"'[cH][cH][cH][o][c]', '[c][cH][cH][cH][o]', '[..."
115894,"'[C][CH2][CH2][CH2][CH]', '[c][cH][cH][o][cH]'..."
...,...
154515,"'[CH][CH2][CH2][C](=[CH])[CH3]', '[CH][C]([CH3..."
154516,"'[cH][cH][cH][cH][c]', '[c][n][cH][c]([c])[cH]..."
154517,"'[C][CH]=[C]([CH2][CH2])[C]([C])([CH3])[CH2]',..."
154518,"'[CH][N]=[N][NH2]', '[CH]=[C]([CH2])[C](=[O])[..."


In [16]:
feature_list3 = []
for x in df_split[3]['Fragments']:
    tup = literal_eval(x)
    feature_list3.append(tup)
feature_list3[-1]

('[c][c]([c])[O][CH3]',
 '[c][c]([CH2])[cH][c]([c])[O]',
 '[c][cH][cH][c]([cH])[CH2]',
 '[c][c]([CH])[c]([CH2][CH2])[c]([c])[OH]',
 '[c][O][c]([c]([c])[CH])[c]([c])[O]',
 '[c][CH]([NH])[CH2][c]([cH])[cH]',
 '[c][O][c]([cH][c])[c]([cH])[O]',
 '[cH][cH][c]([OH])[c]([cH])[O]',
 '[c][c]([c])[O][c]([cH])[c]',
 '[c][CH2][CH2][N]([CH])[CH3]',
 '[c][CH2][CH]([NH][CH2])[c]([c])[cH]',
 '[c][CH]([CH2])[NH][CH2][CH2]',
 '[cH][cH][c]([cH][cH])[CH2][CH]',
 '[c][CH2][CH2][NH][CH]',
 '[c][c]([c])[CH2][CH2][N]',
 '[c][CH2][CH]([c]([c])[c])[N]([CH3])[CH2]',
 '[c][CH]([N])[CH2][c]([cH])[cH]',
 '[cH][c]([CH2])[cH][c]([c])[O]',
 '[c][O][c]([cH][c])[c]([cH])[OH]',
 '[c][O][c]([c]([c])[O])[c]([c])[O]',
 '[c][CH]([CH2])[N]([CH3])[CH2][CH2]',
 '[cH][c]([c])[CH2][CH2][NH]',
 '[c][c]([CH2])[c]([OH])[c]([c])[O]',
 '[c][c]([c])[O][c]([c])[cH]',
 '[c][cH][c]([CH2][CH2])[c]([cH])[CH]',
 '[c][cH][cH][c]([c])[OH]',
 '[c][c]([OH])[c]([O][CH3])[c]([c])[O]',
 '[c][cH][c]([c]([cH])[CH2])[CH]([CH2])[NH]',
 '[c][c]([CH2])[c

In [17]:
feature_list3[0]

('[c][c]([c])[O][CH3]',
 '[cH][c]([O])[c]([OH])[c]([c])[O]',
 '[cH][cH][cH][cH][cH]',
 '[CH2][CH]([CH])[O][CH]([CH])[O]',
 '[c][cH][c]([o][c])-[c]([cH])[cH]',
 '[cH][cH][c]([cH][cH])-[c]([cH])[o]',
 '[c][c]([OH])[c]([O][CH3])[c]([c])[c]',
 '[CH][CH]([O])[CH2][OH]',
 '[c]-[c]([cH])[cH][cH][cH]',
 '[c][cH][c]([O][CH])[c]([c])[OH]',
 '[c][O][CH]([O][CH])[CH]([CH])[OH]',
 '[c]-[c]([o])[cH][c]([c])=[O]',
 '[CH][O][CH]([CH2][OH])[CH]([CH])[OH]',
 '[c][cH][c]([o][c])[c]([c])[c]',
 '[c][cH][c](=[O])[c]([c])[c]',
 '[CH][CH]([OH])[CH]([OH])[CH]([CH])[OH]',
 '[c][c]([cH])[O][CH]([CH])[O]',
 '[CH][CH]([OH])[CH]([OH])[CH]([O])[O]',
 '[c]-[c]([cH])[o][c]([cH])[c]',
 '[c][cH][cH][cH][cH]',
 '[c][c]([O])[c]([c]([cH])=[O])[c]([cH])[o]',
 '[CH2][CH]([O])[CH]([OH])[CH]([CH])[OH]',
 '[c][c]([O])[cH][c]([c])[o]')

In [19]:
p3 = CooccurrenceProfile.from_feature_lists(feature_list3)
p3.df

value
feature1                                feature2                                     
[CH2][CH]([CH])[O][CH]([CH])[O]         [CH2][CH]([CH])[O][CH]([CH])[O]          3352
                                        [CH2][CH]([O])[CH]([OH])[CH]([CH])[OH]   2779
                                        [CH][CH]([OH])[CH]([OH])[CH]([CH])[OH]   2742
                                        [CH][CH]([OH])[CH]([OH])[CH]([O])[O]     2932
                                        [CH][CH]([O])[CH2][OH]                   2616
...                                                                               ...
[c][c]([CH])[c]([CH2][CH2])[c]([c])[OH] [c][c]([c])[O][c]([c])[cH]                  1
[c][c]([OH])[c]([O][CH3])[c]([c])[O]    [c][c]([cH])[O][c]([cH])[cH]                1
                                        [c][c]([c])[O][c]([cH])[c]                  1
                                        [c][c]([c])[O][c]([c])[cH]                  1
[c][c]([c])[CH2][CH2][N]                [c][c]([c])[O][c]([c])[cH]                  1

[4841889 rows x 1 columns]

In [20]:
p3.df.to_csv('../data/CPF/COCONUT/COCOp3_38630.csv')

In [21]:
p3 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp3_38630.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38630)
p3.df

value
feature1                                feature2                                     
[CH2][CH]([CH])[O][CH]([CH])[O]         [CH2][CH]([CH])[O][CH]([CH])[O]          3352
                                        [CH2][CH]([O])[CH]([OH])[CH]([CH])[OH]   2779
                                        [CH][CH]([OH])[CH]([OH])[CH]([CH])[OH]   2742
                                        [CH][CH]([OH])[CH]([OH])[CH]([O])[O]     2932
                                        [CH][CH]([O])[CH2][OH]                   2616
...                                                                               ...
[c][c]([CH])[c]([CH2][CH2])[c]([c])[OH] [c][c]([c])[O][c]([c])[cH]                  1
[c][c]([OH])[c]([O][CH3])[c]([c])[O]    [c][c]([cH])[O][c]([cH])[cH]                1
                                        [c][c]([c])[O][c]([cH])[c]                  1
                                        [c][c]([c])[O][c]([c])[cH]                  1
[c][c]([c])[CH2][CH2][N]                [c][c]([c])[O][c]([c])[cH]                  1

[4841889 rows x 1 columns]

### Coocurence profile 4

In [22]:
df_split[4] = df_split[4] [['Fragments']]
df_split[4]

,Fragments
154520,"'[c][cH][c]([cH][cH])[CH]([CH])[O]', '[cH][cH]..."
154521,'[CH2][CH2][CH]([CH]([CH2])[CH])[C]([CH])([CH2...
154522,"'[C][CH2][CH2][C]([CH2])([CH2])[CH]', '[C][CH2..."
154523,"'[cH][cH][c]([OH])[cH][cH]', '[C]=[CH][CH2][c]..."
154524,"'[C][CH]=[CH][C]([CH3])([CH3])[C]', '[C][CH2][..."
...,...
193145,'[C][C]([C])([OH])[C]([CH3])([CH2][CH2])[CH]([...
193146,"'[CH2][CH]=[C]([C]#[N])[CH2][O]', '[CH2][CH]([..."
193147,"'[cH][c]([cH])[S](=[O])(=[O])[OH]', '[cH][cH][..."
193148,"'[c][c]([c])[O][CH3]', '[c][cH][cH][c]([c])[O]..."


In [23]:
feature_list4 = []
for x in df_split[4]['Fragments']:
    tup = literal_eval(x)
    feature_list4.append(tup)
feature_list4[1]

('[CH2][CH2][CH]([CH]([CH2])[CH])[C]([CH])([CH2])[CH3]',
 '[C][CH2][CH2][CH]([CH])[OH]',
 '[CH2][CH2][CH]([CH]([CH2])[CH2])[C]([CH])([CH2])[CH3]',
 '[CH][CH2][CH2][CH]([CH])[C]',
 '[CH][CH2][CH]=[C]([CH])[C]',
 '[CH2][CH]=[C]([CH]([CH])[OH])[C]([CH2])([CH3])[CH]',
 '[CH2][CH2][CH2][CH]([CH2])[CH]',
 '[CH]=[C]([CH])[C]([CH3])([CH2][CH2])[CH]([CH])[CH2]',
 '[CH2][O][S](=[O])(=[O])[O-]',
 '[C][CH2][CH2][CH]([CH])[C]',
 '[C]=[CH][CH2][CH]([CH])[CH]',
 '[CH]=[C]([C])[CH]([OH])[CH]([CH2])[OH]',
 '[CH2][CH2][C]([CH3])([CH]([CH])[CH2])[CH]([CH2])[CH]',
 '[CH2][CH2][CH2][CH]([CH3])[CH3]',
 '[C][C]([CH3])([CH])[CH2][CH2][CH]',
 '[CH][CH2][CH2][CH2][CH]',
 '[C][C]([CH2])([CH3])[CH]([CH2][CH2])[CH]([CH])[CH2]',
 '[CH2][CH2][CH]([CH3])[CH3]',
 '[CH2][CH]([CH])[CH2][O][S]',
 '[CH][CH2][O][S](=[O])(=[O])[O-]',
 '[CH2][CH2][CH]([CH2][O])[CH]([CH2])[C]',
 '[C][CH]([OH])[CH]([OH])[CH2][CH2]',
 '[CH][CH2][CH]([CH]([C])[CH2])[CH]([CH2])[C]',
 '[CH][CH2][CH2][C]([CH])([CH])[CH3]')

In [24]:
feature_list4[-1]

('[C][C]([CH3])([CH])[CH2][CH2][CH2]',
 '[C][C]([CH2])([OH])[CH]([CH2][C])[CH2][CH2]',
 '[CH2][CH2][CH2][C]([CH])([C])[OH]',
 '[C][C](=[O])[C]([OH])([CH2][OH])[CH]([CH2])[CH2]',
 '[C][C]([CH2])([CH2])[C]([CH3])([CH2][CH2])[C]([CH2])([CH])[OH]',
 '[C][C]([CH2])([C])[CH2][CH2][CH]',
 '[CH3][O][C](=[O])[C]([CH3])([CH2])[CH]',
 '[C][C]([CH2])([C])[CH2][CH]([CH2])[C]',
 '[CH2][CH2][C]([OH])([CH]([C])[CH2])[C]([C])([CH2])[CH3]',
 '[C][CH2][CH2][CH]([CH2])[C]',
 '[CH2][CH2][C]([CH3])([C](=[O])[O])[CH]([C])[CH2]',
 '[CH][CH2][CH2][C]([C])([C])[CH3]',
 '[C][CH2][CH2][CH]([C])[C]',
 '[C][CH2][CH2][CH2][C]',
 '[CH3][O][C]([C])=[O]',
 '[CH2][C]([CH2])([C])[C](=[O])[C]([CH])([CH2])[OH]',
 '[C][C]([CH])([OH])[CH2][OH]',
 '[C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])([C])[OH]',
 '[CH2][CH2][C]([CH2][CH])([C]([C])=[O])[C]([C])([CH2])[CH3]')

In [25]:
p4 = CooccurrenceProfile.from_feature_lists(feature_list4)
p4.df

value
feature1                                           feature2                                                 
[cH][cH][c]([OH])[c]([cH])[O]                      [cH][cH][c]([OH])[c]([cH])[O]                         897
                                                   [cH][cH][c]([cH][c])[CH]([CH])[O]                      45
                                                   [cH][c]([CH])[cH][c]([c])[O]                          295
                                                   [cH][c]([cH])[CH]([O][CH])[CH]([CH3])[CH]               3
                                                   [cH][c]([cH])[CH]([O][CH])[CH]([CH])[CH3]               3
...                                                                                                      ...
[C][C]([CH2])([C])[CH2][CH]([CH2])[C]              [C][C]([CH])([OH])[CH2][OH]                             1
[C][C]([CH2])([OH])[CH]([CH2][C])[CH2][CH2]        [C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(...      1
[C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(... [C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(...      1
                                                   [C][C]([CH3])([CH])[CH2][CH2][CH2]                      1
                                                   [C][C]([CH])([OH])[CH2][OH]                             1

[4821389 rows x 1 columns]

In [26]:
p4.df.to_csv('../data/CPF/COCONUT/COCOp4_38630.csv')

In [27]:
p4 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp4_38630.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38630)
p4.df

value
feature1                                           feature2                                                 
[cH][cH][c]([OH])[c]([cH])[O]                      [cH][cH][c]([OH])[c]([cH])[O]                         897
                                                   [cH][cH][c]([cH][c])[CH]([CH])[O]                      45
                                                   [cH][c]([CH])[cH][c]([c])[O]                          295
                                                   [cH][c]([cH])[CH]([O][CH])[CH]([CH3])[CH]               3
                                                   [cH][c]([cH])[CH]([O][CH])[CH]([CH])[CH3]               3
...                                                                                                      ...
[C][C]([CH2])([C])[CH2][CH]([CH2])[C]              [C][C]([CH])([OH])[CH2][OH]                             1
[C][C]([CH2])([OH])[CH]([CH2][C])[CH2][CH2]        [C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(...      1
[C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(... [C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(...      1
                                                   [C][C]([CH3])([CH])[CH2][CH2][CH2]                      1
                                                   [C][C]([CH])([OH])[CH2][OH]                             1

[4821389 rows x 1 columns]

### Coocurence profile 5

In [28]:
df_split[5] = df_split[5] [['Fragments']]
df_split[5]

,Fragments
193150,"'[CH][CH2][O][C]([CH2])=[O]', '[CH][CH2][CH2][..."
193151,"'[cH][cH][cH][cH][c]', '[C][NH][c]([c]([c])[S]..."
193152,"'[c][cH][cH][cH][c]', '[c]-[c]([c])[c]([CH3])[..."
193153,"'[cH][cH][c]([OH])[cH][cH]', '[c][cH][cH][c]([..."
193154,"'[CH2][C]([CH2])([CH])[C](=[O])[OH]', '[CH]=[C..."
...,...
231775,"'[CH][CH2][O][C]([CH2])=[O]', '[CH][CH]=[CH][C..."
231776,"'[cH][cH][cH][cH][c]', '[cH][cH][c]([NH2])[c](..."
231777,"'[cH][cH][c]([O][CH3])[c]([cH])[NH]', '[c][cH]..."
231778,"'[CH2][CH2][CH2][CH2][CH3]', '[C][CH2][CH2][CH..."


In [29]:
feature_list5 = []
for x in df_split[5]['Fragments']:
    tup = literal_eval(x)
    feature_list5.append(tup)
feature_list5[-1]

('[C][CH2][CH]([CH3])[CH]([CH])[O]',
 '[CH2]=[C]([C](=[O])[O])[CH]([CH])[CH]',
 '[CH][C](=[O])[CH2][CH]([CH3])[CH]',
 '[C][C](=[O])[O][CH]([CH])[CH]',
 '[C][C](=[CH2])[CH]([CH]([CH2])[O])[CH]([CH])[OH]',
 '[CH][O][C](=[O])[C]([CH3])([CH])[O]',
 '[C][CH]([CH])[CH]([OH])[CH]([CH])[O]',
 '[CH]=[C]([CH3])[CH2][CH]([CH])[O]',
 '[C][O][CH]([CH]([CH2])[CH3])[CH]([CH])[OH]',
 '[CH2]=[C]([CH])[C](=[O])[O][CH]',
 '[C][CH2][CH]([O][C])[CH]([C])[CH]',
 '[CH3][CH]1[O][C]1([CH3])[C](=[O])[O]',
 '[C][C]1([CH3])[O][CH]1[CH3]',
 '[C]=[CH][C](=[O])[CH2][CH]',
 '[CH2][C](=[O])[CH]=[C]([CH2])[CH3]',
 '[C][C](=[O])[O][CH]([CH2])[CH]',
 '[C][CH]=[C]([CH3])[CH2][CH]')

In [30]:
feature_list5[0]

('[CH][CH2][O][C]([CH2])=[O]',
 '[CH][CH2][CH2][CH2][CH2]',
 '[CH3][CH2][CH2][CH2]',
 '[CH2][CH2][CH2][CH2][CH2]',
 '[CH2][CH]=[CH][CH2][CH2]',
 '[CH][CH2][CH]=[CH][CH2]',
 '[CH]=[CH][CH2][CH2][CH2]',
 '[CH2][CH2][C](=[O])[O][CH2]',
 '[CH3][CH2][CH2][CH2][CH2]',
 '[CH2][CH]([O])[CH2][OH]',
 '[CH]=[CH][CH2][CH]=[CH]',
 '[C][O][CH]([CH2][O])[CH2][OH]',
 '[CH2][CH2][CH2][CH2][CH3]',
 '[CH2][CH2][CH2][C](=[O])[O]',
 '[C][CH2][CH2][CH2][CH2]',
 '[CH2][CH2][C](=[O])[O][CH]',
 '[C][O][CH2][CH]([CH2])[O]',
 '[CH2][C](=[O])[O][CH]([CH2])[CH2]',
 '[CH2][CH2][CH2][CH3]')

In [31]:
p5 = CooccurrenceProfile.from_feature_lists(feature_list5)
p5.df

value
feature1                                         feature2                                         
[CH2][CH2][CH2][CH2][CH2]                        [CH2][CH2][CH2][CH2][CH2]                    3574
                                                 [CH2][CH2][CH2][CH2][CH3]                     880
                                                 [CH2][CH2][CH2][CH3]                          900
                                                 [CH2][CH2][CH2][C](=[O])[O]                   889
                                                 [CH2][CH2][C](=[O])[O][CH2]                   603
...                                                                                            ...
[C][C](=[CH2])[CH]([CH]([CH2])[O])[CH]([CH])[OH] [C][C]1([CH3])[O][CH]1[CH3]                     1
                                                 [C][O][CH]([CH]([CH2])[CH3])[CH]([CH])[OH]      1
[C][C](=[O])[O][CH]([CH2])[CH]                   [C][O][CH]([CH]([CH2])[CH3])[CH]([CH])[OH]      1
[C][C](=[O])[O][CH]([CH])[CH]                    [C][O][CH]([CH]([CH2])[CH3])[CH]([CH])[OH]      1
[C][C]1([CH3])[O][CH]1[CH3]                      [C][O][CH]([CH]([CH2])[CH3])[CH]([CH])[OH]      1

[4823429 rows x 1 columns]

In [32]:
p5.df.to_csv('../data/CPF/COCONUT/COCOp5_38630.csv')

In [33]:
p5 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp5_38630.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38630)
p5.df

value
feature1                                         feature2                                         
[CH2][CH2][CH2][CH2][CH2]                        [CH2][CH2][CH2][CH2][CH2]                    3574
                                                 [CH2][CH2][CH2][CH2][CH3]                     880
                                                 [CH2][CH2][CH2][CH3]                          900
                                                 [CH2][CH2][CH2][C](=[O])[O]                   889
                                                 [CH2][CH2][C](=[O])[O][CH2]                   603
...                                                                                            ...
[C][C](=[CH2])[CH]([CH]([CH2])[O])[CH]([CH])[OH] [C][C]1([CH3])[O][CH]1[CH3]                     1
                                                 [C][O][CH]([CH]([CH2])[CH3])[CH]([CH])[OH]      1
[C][C](=[O])[O][CH]([CH2])[CH]                   [C][O][CH]([CH]([CH2])[CH3])[CH]([CH])[OH]      1
[C][C](=[O])[O][CH]([CH])[CH]                    [C][O][CH]([CH]([CH2])[CH3])[CH]([CH])[OH]      1
[C][C]1([CH3])[O][CH]1[CH3]                      [C][O][CH]([CH]([CH2])[CH3])[CH]([CH])[OH]      1

[4823429 rows x 1 columns]

### Coocurence profile 6

In [34]:
df_split[6] = df_split[6] [['Fragments']]
df_split[6]

,Fragments
231780,"'[cH][cH][cH][cH][c]', '[CH2][NH][C](=[O])[NH]..."
231781,"'[cH][cH][cH][cH][cH]', '[C][CH]([C])[C]([CH3]..."
231782,'[CH][CH2][CH]([C]([CH3])([CH2])[CH2])[C]([CH2...
231783,"'[cH][cH][c]([OH])[c]([cH])[OH]', '[C][CH]([O]..."
231784,"'[CH2][CH]([CH2])[O][CH]([CH])[O]', '[CH][O][C..."
...,...
270405,"'[cH][c]([cH])[C](=[O])[CH2][CH2]', '[c][cH][c..."
270406,"'[c][c-]([CH])[cH][cH][n]', '[c][O][CH]([CH2][..."
270407,'[C][C](=[CH])[CH]([C]([CH2])=[O])[C]([CH])([C...
270408,"'[cH][cH][c]([cH][cH])[CH]([CH])[N]', '[cH][cH..."


In [35]:
feature_list6 = []
for x in df_split[6]['Fragments']:
    tup = literal_eval(x)
    feature_list6.append(tup)
feature_list6[-1]

('[C][CH]([CH2])[C]([CH3])([CH2][CH2])[CH]([CH2])[C]',
 '[C][CH]=[C]([CH2][CH2])[CH2][O]',
 '[CH2]=[C]([CH])[CH2][CH2][CH]',
 '[C][CH2][CH2][CH]([C])[C]',
 '[CH2][CH2][CH2][C]([CH])([CH3])[CH]',
 '[C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])([CH3])[CH]',
 '[C][CH2][O][C]([CH])=[O]',
 '[C][CH2][CH2][CH2][C]',
 '[CH][C]([CH3])([CH2])[C](=[O])[OH]',
 '[CH2][C]([CH2])=[CH][C](=[O])[O]',
 '[C]=[CH][C](=[O])[O][CH2]',
 '[C][C]([CH])([CH3])[CH2][CH2][CH2]',
 '[C][O][CH2][C](=[CH])[CH2]',
 '[CH]=[C]([CH2])[CH2][CH2][CH]',
 '[CH2]=[C]([CH2][CH2])[CH]([C])[CH2]',
 '[CH2]=[C]([CH2])[CH]([CH2][CH2])[C]([CH])([CH2])[CH3]',
 '[CH2][CH2][C]([CH3])([C](=[O])[OH])[CH]([CH2])[C]')

In [36]:
feature_list6[0]

('[cH][cH][cH][cH][c]',
 '[CH2][NH][C](=[O])[NH][CH]',
 '[c][CH2][CH2][NH][C]',
 '[c][c]([cH])[c]([CH2][CH2])[c]([CH])[nH]',
 '[c][cH][cH][c]([cH])[CH2]',
 '[c][cH][cH][c]([c])[O]',
 '[cH][cH][c]([c]([c])[CH2])[c]([cH])[nH]',
 '[c][c]([c])[CH2][CH2][CH2]',
 '[cH][cH][cH][c]([c])[nH]',
 '[cH][c]([c])[O][CH3]',
 '[c][c]([cH])[O][CH3]',
 '[c][nH][c]([c]([c])[CH2])[CH]([CH2])[NH]',
 '[cH][c]([CH2])[cH][c]([c])[O]',
 '[c][CH]([NH])[CH2][CH2][CH2]',
 '[C][NH][CH]([CH2][CH2])[c]([c])[nH]',
 '[CH2][CH2][NH][C]([NH])=[O]',
 '[cH][cH][c]([O][CH3])[c]([cH])[O]',
 '[cH][cH][c]([nH][c])[c]([c])[cH]',
 '[c][c]([CH])[nH][c]([cH])[c]',
 '[c][CH2][CH2][CH2][CH]',
 '[c][CH]([CH2])[NH][C]([NH])=[O]',
 '[c][cH][cH][cH][cH]',
 '[c][cH][c]([O][CH3])[c]([cH])[O]',
 '[cH][c]([cH])[CH2][CH2][NH]',
 '[cH][cH][cH][c]([c])[c]',
 '[cH][cH][c]([cH][c])[CH2][CH2]')

In [37]:
p6 = CooccurrenceProfile.from_feature_lists(feature_list6)
p6.df

value
feature1                                           feature2                                                 
[CH2][CH2][NH][C]([NH])=[O]                        [CH2][CH2][NH][C]([NH])=[O]                            20
                                                   [CH2][NH][C](=[O])[NH][CH]                             10
                                                   [C][NH][CH]([CH2][CH2])[c]([c])[nH]                     1
                                                   [cH][cH][cH][cH][c]                                     3
                                                   [cH][cH][cH][c]([c])[c]                                 2
...                                                                                                      ...
[C][CH2][O][C]([CH])=[O]                           [C][C]([CH])([CH3])[CH2][CH2][CH2]                      1
[C][CH]=[C]([CH2][CH2])[CH2][O]                    [C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(...      1
                                                   [C][C]([CH])([CH3])[CH2][CH2][CH2]                      1
[C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(... [C][O][CH2][C](=[CH])[CH2]                              1
[C][C]([CH])([CH3])[CH2][CH2][CH2]                 [C][O][CH2][C](=[CH])[CH2]                              1

[4826808 rows x 1 columns]

In [38]:
p6.df.to_csv('../data/CPF/COCONUT/COCOp6_38630.csv')

In [39]:
p6 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp6_38630.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38630)
p6.df

value
feature1                                           feature2                                                 
[CH2][CH2][NH][C]([NH])=[O]                        [CH2][CH2][NH][C]([NH])=[O]                            20
                                                   [CH2][NH][C](=[O])[NH][CH]                             10
                                                   [C][NH][CH]([CH2][CH2])[c]([c])[nH]                     1
                                                   [cH][cH][cH][cH][c]                                     3
                                                   [cH][cH][cH][c]([c])[c]                                 2
...                                                                                                      ...
[C][CH2][O][C]([CH])=[O]                           [C][C]([CH])([CH3])[CH2][CH2][CH2]                      1
[C][CH]=[C]([CH2][CH2])[CH2][O]                    [C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(...      1
                                                   [C][C]([CH])([CH3])[CH2][CH2][CH2]                      1
[C][C]([CH3])([CH2])[CH]([CH2][CH2])[C]([CH2])(... [C][O][CH2][C](=[CH])[CH2]                              1
[C][C]([CH])([CH3])[CH2][CH2][CH2]                 [C][O][CH2][C](=[CH])[CH2]                              1

[4826808 rows x 1 columns]

### Coocurence profile 7

In [40]:
df_split[7] = df_split[7] [['Fragments']]
df_split[7]

,Fragments
270410,"'[CH2][CH2][CH2][CH]([CH])[N]', '[CH][CH2][CH2..."
270411,"'[cH][cH][c]([O][CH3])[c]([c])[CH2]', '[cH][c]..."
270412,"'[cH][cH][c]([cH][cH])-[c]([c])[n]', '[cH][cH]..."
270413,"'[C][c]([c])[cH][cH][c]', '[C][c]([cH])[c]([OH..."
270414,"'[c][c]([c])[CH2][O][CH]', '[cH][cH][cH][cH][c..."
...,...
309034,"'[cH][cH][cH][cH][cH]', '[cH][cH][c]([cH][cH])..."
309035,"'[CH2][CH]([CH])[O][CH]([CH])[O]', '[cH][cH][c..."
309036,"'[CH3][C](=[O])[O][CH]([CH])[CH]', '[CH2][CH2]..."
309037,'[CH][CH2][CH]([C]([C])([CH3])[CH2])[C]([CH])(...


In [41]:
feature_list7 = []
for x in df_split[7]['Fragments']:
    tup = literal_eval(x)
    feature_list7.append(tup)
feature_list7[-1]

('[CH][CH]([OH])[CH]([OH])[CH2][OH]',
 '[CH][CH]([OH])[CH]=[O]',
 '[CH][CH]([OH])[CH]([NH2])[CH]([CH])[OH]',
 '[CH][CH]([NH2])[CH]([OH])[CH]=[O]',
 '[CH][CH]([NH2])[CH]([OH])[CH]([CH2])[OH]',
 '[CH][CH]([OH])[CH2][OH]')

In [42]:
feature_list7[0]

('[CH2][CH2][CH2][CH]([CH])[N]',
 '[CH][CH2][CH2][CH2][CH2]',
 '[C][C]([CH])([CH2])[CH2][N]([CH])[CH2]',
 '[CH2][N]=[C]([CH]([CH2])[CH2])[C]([CH])([CH2])[CH2]',
 '[CH][CH2][C]([CH2][N])([C]([CH])=[N])[CH]([CH2])[NH]',
 '[CH][CH2][CH]([CH2][CH2])[C]([C])=[N]',
 '[C][C]([CH2])([CH2])[CH]([CH2][CH2])[NH][CH2]',
 '[CH2][CH2][CH2][N]([CH])[CH2]',
 '[C][CH2][N]([CH2][CH2])[CH]([CH])[CH2]',
 '[C][CH2][CH]([CH2][CH])[CH]([CH2])[N]',
 '[CH2][CH2][CH2][CH2][N]',
 '[CH2][CH2][CH2][CH]([C])[NH]',
 '[C]=[N][CH2][CH2][CH2]',
 '[C][C]([CH])([CH2])[CH2][CH]([CH2])[CH]',
 '[CH2][CH2][CH2][NH][CH]',
 '[CH2][CH2][N]=[C]([C])[CH]',
 '[CH2][CH2][CH]([CH]([CH2])[CH2])[N]([CH2])[CH2]',
 '[CH2][CH2][CH2][CH2][NH]',
 '[CH2][CH2][NH][CH]([CH2])[C]',
 '[C][CH]([CH2])[CH2][CH2][CH2]',
 '[C][CH]([CH2])[CH2][CH]([CH2])[CH]',
 '[CH][CH2][CH2][CH2][N]')

In [43]:
p7 = CooccurrenceProfile.from_feature_lists(feature_list7)
p7.df

value
feature1                                 feature2                                      
[CH2][CH2][CH2][CH2][NH]                 [CH2][CH2][CH2][CH2][NH]                   257
                                         [CH2][CH2][CH2][CH2][N]                     31
                                         [CH2][CH2][CH2][CH]([CH])[N]                 5
                                         [CH2][CH2][CH2][CH]([C])[NH]                 1
                                         [CH2][CH2][CH2][NH][CH]                     25
...                                                                                 ...
[CH][CH]([NH2])[CH]([OH])[CH]([CH2])[OH] [CH][CH]([OH])[CH]([NH2])[CH]([CH])[OH]      1
                                         [CH][CH]([OH])[CH]([OH])[CH2][OH]            1
                                         [CH][CH]([OH])[CH]=[O]                       1
[CH][CH]([NH2])[CH]([OH])[CH]=[O]        [CH][CH]([OH])[CH2][OH]                      1
                                         [CH][CH]([OH])[CH]([OH])[CH2][OH]            1

[4858596 rows x 1 columns]

In [44]:
p7.df.to_csv('../data/CPF/COCONUT/COCOp7_38629.csv')

In [45]:
p7 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp7_38629.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38629)
p7.df

value
feature1                                 feature2                                      
[CH2][CH2][CH2][CH2][NH]                 [CH2][CH2][CH2][CH2][NH]                   257
                                         [CH2][CH2][CH2][CH2][N]                     31
                                         [CH2][CH2][CH2][CH]([CH])[N]                 5
                                         [CH2][CH2][CH2][CH]([C])[NH]                 1
                                         [CH2][CH2][CH2][NH][CH]                     25
...                                                                                 ...
[CH][CH]([NH2])[CH]([OH])[CH]([CH2])[OH] [CH][CH]([OH])[CH]([NH2])[CH]([CH])[OH]      1
                                         [CH][CH]([OH])[CH]([OH])[CH2][OH]            1
                                         [CH][CH]([OH])[CH]=[O]                       1
[CH][CH]([NH2])[CH]([OH])[CH]=[O]        [CH][CH]([OH])[CH2][OH]                      1
                                         [CH][CH]([OH])[CH]([OH])[CH2][OH]            1

[4858596 rows x 1 columns]

#### Coocurence profile 8

In [46]:
df_split[8] = df_split[8] [['Fragments']]
df_split[8]

,Fragments
309039,"'[CH]=[C]([CH])[CH2][CH2][NH+]', '[C][CH2][CH2..."
309040,"'[c]-[c]([cH])[cH][c]([c])[O]', '[cH][c]([CH])..."
309041,"'[cH][cH][cH][cH][c]', '[cH][cH][cH][cH][cH]',..."
309042,"'[c][c]([c])[O][CH3]', '[c][c]([CH2])[cH][c]([..."
309043,"'[cH][cH][c]([OH])[cH][cH]', '[cH][cH][cH][cH]..."
...,...
347663,'[C][CH]=[C]([C]([c])([CH3])[CH2])[C]([CH2])([...
347664,"'[cH][c](=[O])[o][c]([cH])[c]', '[c][c]([c])[O..."
347665,"'[c][cH][c]([CH3])[c]([c])[OH]', '[C][C](=[C])..."
347666,"'[C][CH]([NH2])[CH]([OH])[C]([NH2])=[O]', '[C]..."


In [47]:
feature_list8 = []
for x in df_split[8]['Fragments']:
    tup = literal_eval(x)
    feature_list8.append(tup)
feature_list8[-1]

('[C][NH][C](=[O])[C]([CH3])([CH])[CH]',
 '[C][NH][C](=[O])[NH][CH]',
 '[CH][C]([NH2])=[N][C]([NH])=[O]',
 '[C][C](=[O])[NH][C]([NH])=[O]',
 '[C][NH][CH]1[CH]([NH])[CH][C]1([C])[CH3]',
 '[C][N]=[C]([NH2])[CH]([C])[CH]',
 '[CH][CH]([CH])[NH][C]([N])=[O]',
 '[C][C]1([CH3])[CH][CH]([NH])[CH]1[C](=[N])[NH2]',
 '[C][CH]([CH])[NH][C]([NH])=[O]',
 '[C]=[N][C](=[O])[NH][CH]',
 '[C][NH][CH]1[CH]([C])[C][CH]1[NH]',
 '[C][CH]1[CH][CH]([NH])[C]1([CH3])[C]([NH])=[O]')

In [48]:
feature_list8[0]

('[CH]=[C]([CH])[CH2][CH2][NH+]',
 '[C][CH2][CH2][NH+]([CH3])[CH]',
 '[c][C](=[O])[O][CH]([CH])[CH2]',
 '[C]=[CH][CH2][CH]([CH])[O]',
 '[cH][c]([c])[C](=[O])[O][CH]',
 '[c][cH][c]([C](=[O])[O])[c]([cH])[CH]',
 '[C][c]([cH])[c]([cH][c])[CH]([CH])[CH]',
 '[C][CH]([CH])[NH+]([CH3])[CH2][CH2]',
 '[cH][c]([c])[O][CH3]',
 '[c][c]([cH])[O][CH3]',
 '[c][CH]([CH])[CH]([C](=[CH])[CH2])[NH+]([CH2])[CH3]',
 '[CH][CH2][CH]=[C]([CH2])[CH]',
 '[CH2][CH]=[C]([CH2][CH2])[CH]([CH])[NH+]',
 '[c][c]([CH])[cH][c]([c])[O]',
 '[C][c]([c])[cH][c]([c])[O]',
 '[C][CH]([NH+])[CH]([c]([c])[cH])[CH]([CH2])[O]',
 '[CH][CH2][CH]([O][C])[CH]([c])[CH]',
 '[c][cH][c]([O][CH3])[c]([cH])[O]')

In [49]:
p8 = CooccurrenceProfile.from_feature_lists(feature_list8)
p8.df

value
feature1                                 feature2                                       
[CH2][CH]=[C]([CH2][CH2])[CH]([CH])[NH+] [CH2][CH]=[C]([CH2][CH2])[CH]([CH])[NH+]      1
                                         [CH]=[C]([CH])[CH2][CH2][NH+]                 1
                                         [CH][CH2][CH]([O][C])[CH]([c])[CH]            1
                                         [CH][CH2][CH]=[C]([CH2])[CH]                  1
                                         [C]=[CH][CH2][CH]([CH])[O]                    1
...                                                                                  ...
[C][NH][CH]1[CH]([NH])[CH][C]1([C])[CH3] [C][NH][C](=[O])[C]([CH3])([CH])[CH]          1
                                         [C][NH][C](=[O])[NH][CH]                      1
                                         [C][N]=[C]([NH2])[CH]([C])[CH]                1
[C][NH][C](=[O])[C]([CH3])([CH])[CH]     [C][NH][C](=[O])[NH][CH]                      1
[C][NH][C](=[O])[NH][CH]                 [C][N]=[C]([NH2])[CH]([C])[CH]                1

[4809803 rows x 1 columns]

In [50]:
p8.df.to_csv('../data/CPF/COCONUT/COCOp8_38629.csv')

In [51]:
p8 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp8_38629.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38629)
p8.df

value
feature1                                 feature2                                       
[CH2][CH]=[C]([CH2][CH2])[CH]([CH])[NH+] [CH2][CH]=[C]([CH2][CH2])[CH]([CH])[NH+]      1
                                         [CH]=[C]([CH])[CH2][CH2][NH+]                 1
                                         [CH][CH2][CH]([O][C])[CH]([c])[CH]            1
                                         [CH][CH2][CH]=[C]([CH2])[CH]                  1
                                         [C]=[CH][CH2][CH]([CH])[O]                    1
...                                                                                  ...
[C][NH][CH]1[CH]([NH])[CH][C]1([C])[CH3] [C][NH][C](=[O])[C]([CH3])([CH])[CH]          1
                                         [C][NH][C](=[O])[NH][CH]                      1
                                         [C][N]=[C]([NH2])[CH]([C])[CH]                1
[C][NH][C](=[O])[C]([CH3])([CH])[CH]     [C][NH][C](=[O])[NH][CH]                      1
[C][NH][C](=[O])[NH][CH]                 [C][N]=[C]([NH2])[CH]([C])[CH]                1

[4809803 rows x 1 columns]

### Coocurence profile 9

In [52]:
df_split[9] = df_split[9] [['Fragments']]
df_split[9]

,Fragments
347668,"'[C][CH]([CH2])[CH]([OH])[CH]([C])[CH]', '[CH]..."
347669,"'[cH][cH][c]([cH][cH])[CH2][CH]', '[cH][cH][cH..."
347670,"'[cH][cH][cH][o][c]', '[c][cH][cH][cH][o]', '[..."
347671,"'[cH][c]([cH])[CH2][CH2][C]', '[C][CH]([CH2])[..."
347672,"'[cH][cH][cH][cH][c]', '[c][C](=[C])[O][c]([c]..."
...,...
386292,"'[CH]=[C]([CH])[C](=[O])[O][CH3]', '[CH2][CH](..."
386293,"'[NH][CH2][C](=[O])[O-]', '[c][cH][c]([Cl])[c]..."
386294,"'[c][cH][c]([CH3])[c]([c])[O]', '[C][C](=[CH])..."
386295,"'[CH2][CH]([CH])[O][CH]([CH])[O]', '[CH][O][CH..."


In [53]:
feature_list9 = []
for x in df_split[9]['Fragments']:
    tup = literal_eval(x)
    feature_list9.append(tup)
feature_list9[-1]

('[c][cH][cH][cH][c]',
 '[C][C]([C])([OH])[CH]([CH2][CH2])[O][CH]',
 '[C][c]([c])[cH][cH][cH]',
 '[cH][cH][cH][c]([c])[OH]',
 '[C][C]([C])([OH])[CH2][CH]([CH2])[O]',
 '[c][c]([c])[C](=[O])[C]([CH])([C])[OH]',
 '[CH2][CH]([CH2])[O][CH]([CH2])[C]',
 '[CH3][CH2][CH2][CH]',
 '[c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([CH2])[OH]',
 '[C][c]([cH])[c]([C]([C])=[O])[c]([cH])[OH]',
 '[CH3][CH2][CH2][CH]([C])[O]',
 '[c][C](=[O])[C]([OH])([CH2][CH])[C]([C])([CH])[OH]',
 '[CH2][CH]([O])[CH2][C](=[O])[OH]',
 '[C][CH2][CH]([CH2][C])[O][CH]',
 '[CH][CH2][C](=[O])[OH]',
 '[cH][c]([c])[C](=[O])[C]([CH2])([C])[OH]',
 '[C][c]([c])[c]([OH])[cH][cH]',
 '[C][c]([c])[c]([cH][cH])[C]([C])=[O]')

In [54]:
feature_list9[0]

('[C][CH]([CH2])[CH]([OH])[CH]([C])[CH]',
 '[CH][CH2][CH]([OH])[CH]([CH3])[CH]',
 '[CH2][CH]([CH])[O][CH]([CH])[O]',
 '[CH][CH2][CH2][C]([CH3])([CH])[CH]',
 '[CH][CH]([O])[CH2][OH]',
 '[CH2]=[C]([C](=[O])[O])[CH]([CH2])[CH]',
 '[C][CH]([CH])[CH2][CH2][C]',
 '[C][CH]([O])[CH2][CH]([CH])[OH]',
 '[CH][CH]([CH3])[CH]([CH]([CH])[OH])[C]([CH2])([CH3])[CH]',
 '[CH][O][CH]([CH2][OH])[CH]([CH])[OH]',
 '[CH2][CH2][C]([CH3])([CH]([CH])[CH])[CH]([CH2])[O]',
 '[CH2]=[C]([CH])[C](=[O])[O][CH3]',
 '[C][CH]([CH2])[O][CH]([CH])[O]',
 '[CH][CH2][CH]([O][CH])[C]([CH2])([CH3])[CH]',
 '[CH][O][CH]([O][CH])[CH]([CH])[OH]',
 '[CH][CH]([OH])[CH]([OH])[CH]([CH])[OH]',
 '[C][CH]([CH])[CH]([CH3])[CH]([CH2])[OH]',
 '[CH][CH]([OH])[CH]([OH])[CH]([O])[O]',
 '[C][C](=[CH2])[CH]([CH2][CH2])[CH]([CH])[OH]',
 '[C][C](=[O])[O][CH3]',
 '[CH2][CH]([O])[CH]([OH])[CH]([CH])[OH]')

In [55]:
p9 = CooccurrenceProfile.from_feature_lists(feature_list9)
p9.df

value
feature1                                           feature2                                                 
[CH2]=[C]([CH])[C](=[O])[O][CH3]                   [CH2]=[C]([CH])[C](=[O])[O][CH3]                       15
                                                   [CH2]=[C]([C](=[O])[O])[CH]([CH2])[CH]                  3
                                                   [CH2][CH2][C]([CH3])([CH]([CH])[CH])[CH]([CH2])[O]      2
                                                   [CH2][CH]([CH])[O][CH]([CH])[O]                         2
                                                   [CH2][CH]([O])[CH]([OH])[CH]([CH])[OH]                  2
...                                                                                                      ...
[cH][c]([c])[C](=[O])[C]([CH2])([C])[OH]           [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[c][C](=[O])[C]([OH])([CH2][CH])[C]([C])([CH])[OH] [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([... [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
                                                   [c][cH][cH][cH][c]                                      1
                                                   [c][c]([c])[C](=[O])[C]([CH])([C])[OH]                  1

[4818575 rows x 1 columns]

In [56]:
p9.df.to_csv('../data/CPF/COCONUT/COCOp9_38629.csv')

In [57]:
p9 = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp9_38629.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=38629)
p9.df

value
feature1                                           feature2                                                 
[CH2]=[C]([CH])[C](=[O])[O][CH3]                   [CH2]=[C]([CH])[C](=[O])[O][CH3]                       15
                                                   [CH2]=[C]([C](=[O])[O])[CH]([CH2])[CH]                  3
                                                   [CH2][CH2][C]([CH3])([CH]([CH])[CH])[CH]([CH2])[O]      2
                                                   [CH2][CH]([CH])[O][CH]([CH])[O]                         2
                                                   [CH2][CH]([O])[CH]([OH])[CH]([CH])[OH]                  2
...                                                                                                      ...
[cH][c]([c])[C](=[O])[C]([CH2])([C])[OH]           [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[c][C](=[O])[C]([OH])([CH2][CH])[C]([C])([CH])[OH] [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([... [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
                                                   [c][cH][cH][cH][c]                                      1
                                                   [c][c]([c])[C](=[O])[C]([CH])([C])[OH]                  1

[4818575 rows x 1 columns]

### Cooccurrence profile whole

In [3]:
#Loading database from csv 
suppl_coco = pd.read_csv('../data/CPF/COCONUT/COCOrandomF.csv', delimiter = ' ')

In [4]:
suppl_coco

,Coconut_id,Smiles,MW,logP,Inchi,Inchi_s,Name,ECFP,Bitset,Fragments
0,CNP0239046,CCCCCC1C(=O)CCC1CC(=O)OC,226.156895,2.72510,KVWWIYGFBYDJQC-UHFFFAOYSA-N,KVWWIYGFBYDJQC,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[80, 142, 195, 226, 294, 314, 334, 339, 370, 5...","'[CH][CH2][CH2][C]([CH])=[O]', '[CH][CH2][C](=..."
1,CNP0205086,Cc1c2c(cc3c1oc(=O)c1ccccc13)CN(c1cccc(Cl)c1)CO2,377.081871,5.26452,DWNKYEMDZRSBHK-UHFFFAOYSA-N,DWNKYEMDZRSBHK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[133, 137, 165, 216, 252, 314, 350, 382, 398, ...","'[c][c]([CH2])[cH][c]([c])[c]', '[cH][cH][cH][..."
2,CNP0361840,COc1cc(C2CC(=O)Oc3ccc4c(=O)c(-c5ccccc5)coc4c32...,458.136553,4.92680,ZRAWZNDFONIPGO-UHFFFAOYSA-N,ZRAWZNDFONIPGO,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[45, 60, 133, 145, 222, 227, 231, 249, 282, 28...","'[c][c]([c])[O][CH3]', '[c][cH][cH][c]([c])[c]..."
3,CNP0264800,C=C1C(=O)OC2C=C(CO)CCC(OCC)C3=CC(OC3=O)C12,320.125988,1.05360,PTJPAEBQCZMNAV-UHFFFAOYSA-N,PTJPAEBQCZMNAV,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[35, 80, 176, 222, 294, 314, 408, 514, 650, 65...","'[CH2][C]([CH2])=[CH][CH]([CH])[O]', '[C][C](=..."
4,CNP0396333,CC(=O)C(CC=C(C)C)C(C)=CCOc1ccc(CCNC(=O)c2ccccc...,419.246044,5.54570,FGXBFPFGWRVMGG-UHFFFAOYSA-N,FGXBFPFGWRVMGG,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[1, 8, 80, 102, 197, 265, 310, 322, 367, 389, ...","'[cH][cH][cH][cH][cH]', '[c][CH2][CH2][NH][C]'..."
...,...,...,...,...,...,...,...,...,...,...
386292,CNP0315857,COC(=O)C1=COC(OC2OC(COC(=O)c3ccc(O)c(OC)c3)C(O...,554.163556,-0.63720,LIVSNGMPJUCRLO-UHFFFAOYSA-N,LIVSNGMPJUCRLO,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[58, 80, 145, 147, 167, 231, 233, 249, 314, 42...","'[CH]=[C]([CH])[C](=[O])[O][CH3]', '[CH2][CH](..."
386293,CNP0032553,Cc1c(CCC(=O)NCC(=O)NCC(=O)[O-])c(=O)oc2cc(O)c(...,395.065152,-0.62468,SSZVGMXOKPZNCV-UHFFFAOYSA-M,SSZVGMXOKPZNCV,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[41, 80, 197, 252, 254, 305, 314, 366, 406, 42...","'[NH][CH2][C](=[O])[O-]', '[c][cH][c]([Cl])[c]..."
386294,CNP0223192,CC(=O)Oc1c(C)cc2c(c1O)C(=O)C(C1=CC(=O)c3cc(C)c...,490.089997,2.87624,JODPOMYHTFZNTK-UHFFFAOYSA-N,JODPOMYHTFZNTK,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[16, 39, 84, 202, 224, 314, 333, 352, 516, 641...","'[c][cH][c]([CH3])[c]([c])[O]', '[C][C](=[CH])..."
386295,CNP0356727,C[N+]([O-])=NCOC1OC(CO)C(O)C(OC2OC(COC3OC(CO)C...,576.201398,-6.99890,STHHNNUYQQUJGY-UHFFFAOYSA-N,STHHNNUYQQUJGY,COCO,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,"[77, 80, 170, 222, 349, 443, 489, 588, 610, 65...","'[CH2][CH]([CH])[O][CH]([CH])[O]', '[CH][O][CH..."


In [5]:
feature_list_all = []
for x in suppl_coco['Fragments']:
    tup = literal_eval(x)
    feature_list_all.append(tup)
feature_list_all[-1]

('[c][cH][cH][cH][c]',
 '[C][C]([C])([OH])[CH]([CH2][CH2])[O][CH]',
 '[C][c]([c])[cH][cH][cH]',
 '[cH][cH][cH][c]([c])[OH]',
 '[C][C]([C])([OH])[CH2][CH]([CH2])[O]',
 '[c][c]([c])[C](=[O])[C]([CH])([C])[OH]',
 '[CH2][CH]([CH2])[O][CH]([CH2])[C]',
 '[CH3][CH2][CH2][CH]',
 '[c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([CH2])[OH]',
 '[C][c]([cH])[c]([C]([C])=[O])[c]([cH])[OH]',
 '[CH3][CH2][CH2][CH]([C])[O]',
 '[c][C](=[O])[C]([OH])([CH2][CH])[C]([C])([CH])[OH]',
 '[CH2][CH]([O])[CH2][C](=[O])[OH]',
 '[C][CH2][CH]([CH2][C])[O][CH]',
 '[CH][CH2][C](=[O])[OH]',
 '[cH][c]([c])[C](=[O])[C]([CH2])([C])[OH]',
 '[C][c]([c])[c]([OH])[cH][cH]',
 '[C][c]([c])[c]([cH][cH])[C]([C])=[O]')

In [6]:
feature_list_all[0]

('[CH][CH2][CH2][C]([CH])=[O]',
 '[CH][CH2][C](=[O])[O][CH3]',
 '[C][CH2][CH2][CH]([CH])[CH2]',
 '[C][CH]([CH])[CH2][CH2][CH2]',
 '[CH][CH]([CH2])[CH2][C](=[O])[O]',
 '[CH3][CH2][CH2][CH2]',
 '[CH3][CH2][CH2][CH2][CH2]',
 '[C][CH2][CH]([CH2][CH2])[CH]([C])[CH2]',
 '[CH2][CH2][C](=[O])[CH]([CH2])[CH]',
 '[CH2][CH2][CH]([C]([CH2])=[O])[CH]([CH2])[CH2]',
 '[CH2][CH2][CH2][CH2][CH]',
 '[CH3][O][C]([CH2])=[O]')

In [7]:
print(len(feature_list_all))

386297


In [8]:
%%time
p_all = CooccurrenceProfile.from_feature_lists(feature_list_all)
p_all.df

CPU times: user 1min 46s, sys: 2.85 s, total: 1min 49s
Wall time: 1min 49s


value
feature1                                           feature2                                                 
[CH2][CH2][CH2][CH2][CH]                           [CH2][CH2][CH2][CH2][CH]                            15582
                                                   [CH2][CH2][CH]([C]([CH2])=[O])[CH]([CH2])[CH2]         11
                                                   [CH2][CH2][C](=[O])[CH]([CH2])[CH]                    139
                                                   [CH3][CH2][CH2][CH2]                                 7133
                                                   [CH3][CH2][CH2][CH2][CH2]                            6905
...                                                                                                      ...
[C][c]([c])[cH][cH][cH]                            [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[C][c]([c])[c]([OH])[cH][cH]                       [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[C][c]([c])[c]([cH][cH])[C]([C])=[O]               [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[cH][cH][cH][c]([c])[OH]                           [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([... [c][cH][cH][cH][c]                                      1

[18368174 rows x 1 columns]

In [9]:
p_all.df.to_csv('../data/CPF/COCONUT/COCOp_386297.csv')

In [10]:
p_all = profiles.CooccurrenceProfile.from_dataframe(pd.read_csv('../data/CPF/COCONUT/COCOp_386297.csv', dtype={'feature1': 'str','feature2' : 'str' }), vector_count=386297)
p_all.df

value
feature1                                           feature2                                                 
[CH2][CH2][CH2][CH2][CH]                           [CH2][CH2][CH2][CH2][CH]                            15582
                                                   [CH2][CH2][CH]([C]([CH2])=[O])[CH]([CH2])[CH2]         11
                                                   [CH2][CH2][C](=[O])[CH]([CH2])[CH]                    139
                                                   [CH3][CH2][CH2][CH2]                                 7133
                                                   [CH3][CH2][CH2][CH2][CH2]                            6905
...                                                                                                      ...
[C][c]([c])[cH][cH][cH]                            [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[C][c]([c])[c]([OH])[cH][cH]                       [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[C][c]([c])[c]([cH][cH])[C]([C])=[O]               [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[cH][cH][cH][c]([c])[OH]                           [c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([...      1
[c][C](=[O])[C]([OH])([CH]([CH2])[O])[C]([C])([... [c][cH][cH][cH][c]                                      1

[18368174 rows x 1 columns]